In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
import re
import pprint
from sklearn.utils import shuffle
import gensim

from gensim import corpora
from gensim.parsing.preprocessing import remove_stopwords
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer


In [3]:
data = pd.read_csv('../input/amazon-questionanswer-dataset/single_qna.csv')

In [4]:
data.head()

,QuestionType,Asin,AnswerTime,UnixTime,Question,AnswerType,Answer,Category
0,yes/no,B00004U9JP,"Jun 27, 2014",1.403852e+09,I have a 9 year old Badger 1 that needs replac...,?,I replaced my old one with this without a hitch.,Appliances
1,open-ended,B00004U9JP,"Apr 28, 2014",1.398668e+09,model number,NaN,This may help InSinkErator Model BADGER-1: Bad...,Appliances
2,yes/no,B00004U9JP,"Aug 25, 2014",1.408950e+09,can I replace Badger 1 1/3 with a Badger 5 1/2...,?,Plumbing connections will vary with different ...,Appliances
3,yes/no,B00004U9JP,"Nov 3, 2014",1.415002e+09,Does this come with power cord and dishwasher ...,?,It does not come with a power cord. It does co...,Appliances
4,open-ended,B00004U9JP,"Jun 21, 2014",1.403334e+09,loud noise inside when turned on. sounds like ...,NaN,Check if you dropped something inside.Usually ...,Appliances


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1396896 entries, 0 to 1396895
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   QuestionType  1396896 non-null  object 
 1   Asin          1396896 non-null  object 
 2   AnswerTime    1396896 non-null  object 
 3   UnixTime      1346991 non-null  float64
 4   Question      1396895 non-null  object 
 5   AnswerType    663866 non-null   object 
 6   Answer        1396798 non-null  object 
 7   Category      1396896 non-null  object 
dtypes: float64(1), object(7)
memory usage: 85.3+ MB


In [6]:
data = data.drop(['QuestionType','Asin', 'AnswerTime','UnixTime','AnswerType'],axis =1)

In [7]:
data.head()

,Question,Answer,Category
0,I have a 9 year old Badger 1 that needs replac...,I replaced my old one with this without a hitch.,Appliances
1,model number,This may help InSinkErator Model BADGER-1: Bad...,Appliances
2,can I replace Badger 1 1/3 with a Badger 5 1/2...,Plumbing connections will vary with different ...,Appliances
3,Does this come with power cord and dishwasher ...,It does not come with a power cord. It does co...,Appliances
4,loud noise inside when turned on. sounds like ...,Check if you dropped something inside.Usually ...,Appliances


In [8]:
data.describe()

,Question,Answer,Category
count,1396895,1396798,1396896
unique,1199374,1162056,21
top,What are the dimensions?,Yes,Electronics
freq,924,33076,314263


In [9]:
data.Category.value_counts()

Electronics                    314263
Home and Kitchen               184439
Sports and Outdoors            146891
Tools and Home Improvement     101088
Automotive                      89923
Cell Phones and Accessories     85865
Health and Personal Care        80496
Patio Lawn and Garden           59595
Toys and Games                  51486
Office Products                 43608
Beauty                          42422
Pet Supplies                    36607
Baby                            28933
Musical Instruments             23322
Clothing Shoes and Jewelry      22068
Arts Crafts and Sewing          21262
Grocery and Gourmet Food        19538
Video Games                     13307
Industrial and Scientific       12136
Software                        10636
Appliances                       9011
Name: Category, dtype: int64

In [10]:
print(data.isnull().sum())
data = data.dropna()

Question     1
Answer      98
Category     0
dtype: int64


In [11]:
data.isnull().sum()

Question    0
Answer      0
Category    0
dtype: int64

In [12]:
digit =[]
for i in range(0,len(data)):
    digit.append(int(i))

In [13]:
digit = pd.DataFrame(digit)

In [14]:
data['index'] = digit

In [15]:
data['index']=data.index.astype('int')

In [16]:
data=data.set_index('index')

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1396797 entries, 0 to 1396895
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   Question  1396797 non-null  object
 1   Answer    1396797 non-null  object
 2   Category  1396797 non-null  object
dtypes: object(3)
memory usage: 42.6+ MB


In [18]:
data =data.reindex(list(range(data.index.min(),data.index.max()+1)),fill_value=0)
#filling missing indexes with zeros so that we donot get any error

In [19]:
#shuffling the data
data = shuffle(data)

In [20]:
data= data.reset_index()
data = data.drop(['index'],axis =1)


In [21]:
#data = data.drop(['Category'],axis =1)

In [22]:
data = data[:100000]  # going to work only with 100000 datset because it took a large time to process all the 1300000 dataset

In [23]:
def processing(sentence,stopwords =False):
    #word =[]
    #for i in range(len(sentence)):
    #word = sentence.lower().strip()
    word = re.sub('^[a-z0-9\s-]'' , ', ' ',str(sentence))
    word = word.lower().strip()
        
        
        #word.append(words)
    #if stopwords:
        #sentence = remove_stopwords(sentence)
    return word   


def get_processed_text(sentence,stopwords =False):
    sents = data[['Question']]
    cleaned_sentences = []
    
    for index,row in data.iterrows():
        cleaned = processing(row['Question'],stopwords=False)
        cleaned_sentences.append(cleaned)
        
    return cleaned_sentences


#print(cleaned_sentences[0])
    

In [24]:
cleaned_sentences = get_processed_text(data,stopwords=False)

In [25]:
sentences = shuffle(cleaned_sentences)
sentences = cleaned_sentences[:8000]  #because my devise is taking too much time for 1,00,000 dataset
#sentences = shuffle(sentences)

In [26]:
len(sentences)

8000

In [27]:

def retreive_and_print_answer(question_embedding,bow_corpus,data_model,word):
    max_sim =-1
    index_sim =-1
    
    for index,embedding_amz in enumerate(bow_corpus):
        sim = cosine_similarity(embedding_amz,question_embedding)[0][0]
        #sim = cosine_similarity(np.array(embedding_amz).reshape(1,-1),np.array(question_embedding).reshape(1,-1))
        #print(index,sim,word[index])
        
    
        if sim>max_sim:
            max_sim=sim
            index_sim =index
            
        
        
    print('\n')
    print("Your Entered Question : ",text)
    print('\n')
    print("Similar Question We Got :  ",data_model.iloc[index_sim,0])
    print('\n')
    print("ANSWER:  ",data_model.iloc[index_sim,1])
    
#retreive_and_print_answer(question_embedding,bow_corpus,data,sentences)
    

In [28]:
!pip install transformers

In [29]:
import torch

In [30]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 561.4 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=281d72c67c5e1324d3726127f55b6b86dadb72944a58001ffdf6f5db6ff3a268
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [31]:
!pip install -U sentence-transformers

In [32]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [33]:
#from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer
#model = SentenceTransformer('all-MiniLM-L6-v2')

In [34]:
from tqdm import tqdm

In [ ]:
model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')


bert_sent = []
for sent in tqdm(sentences):
    bert_sent.append(model.encode([sent],batch_size = 64))



def bertmodel(text):
    processed_text = processing(text,stopwords = False)
    embedding = model.encode([processed_text])
    
    retreive_and_print_answer(embedding,bert_sent,data,sentences)




In [36]:
text = input('please enter a query related to product or general : ')
print('.........result from bert model0 technique......')
bertmodel(text)

please enter a query related to product or general :  my battery is not working properly. how should i replace it?


.........result from bert model0 technique......


Batches:   0%|          | 0/1 [00:00<?, ?it/s]



Your Entered Question :  my battery is not working properly. how should i replace it?


Similar Question We Got :   Is there a way to change the battery? Ours no longer keeps a good charge.


ANSWER:   If there's a way to change the battery, I couldn't find it. Mine died after about 4 months. I like the unit, so bought a new one but if its battery dies as soon I'll have to try another product.
